# Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.model_selection import train_test_split

# adding new libraries
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Flatten 
from tensorflow.keras.layers import Dense, Dropout 
from tensorflow.keras.layers import Activation 
from tensorflow.keras import layers, Input
from tensorflow.keras.regularizers import l1, l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler  # Import MinMaxScaler
from tensorflow.keras.optimizers import Adam
from keras.optimizers import SGD
from tensorflow.keras.losses import MeanAbsoluteError
from keras.regularizers import l2


# Import Data

In [3]:
# df = pd.read_csv('../../res/model_data/batemo_model_data.csv')
# df = df.drop('SOH', axis=1) # Remove SOH from input data
# 
# # add moving averages of V, I, SOC, and T_surface to the input data
# df['V_ma'] = df['V'].rolling(window=10).mean()
# df['I_ma'] = df['I'].rolling(window=10).mean()
# df['T_surf_ma'] = df['T_surf'].rolling(window=10).mean()
# 
# # drop the first 10 rows of the data since the moving averages are NaN for the first 10 rows
# df = df.dropna()
# 
# df

# Import Data
df = pd.read_csv('../../res/model_data/batemo_model_data.csv')
df.describe()


,V,I,SOC,T_surf,SOH
count,1.640946e+06,1.640946e+06,1.640946e+06,1.640946e+06,1.640946e+06
mean,3.471013e+00,3.610140e+00,4.188879e+01,5.180549e+01,8.566437e+01
std,8.445745e-01,2.830966e+01,4.022232e+01,2.451698e+01,9.026790e+00
min,-9.394656e-01,-5.500295e+01,-5.753759e+00,2.499965e+01,7.000000e+01
25%,3.144859e+00,-1.900000e+01,1.230206e+00,2.583835e+01,7.800000e+01
50%,3.693897e+00,5.000000e+00,3.474387e+01,4.826449e+01,8.600000e+01
75%,4.064538e+00,2.600000e+01,8.456010e+01,7.304868e+01,9.400000e+01
max,4.723157e+00,5.500289e+01,1.070423e+02,1.015903e+02,1.000000e+02


# Train Test Split

In [6]:
# Define training variables
# X = df[['V', 'I', 'T_surf']]
# X = df[['V', 'I', 'T_surf', 'SOC']]
X = df[['V', 'I', 'T_surf', 'SOC']].values
# TODO: may need to add SOC as an input feature for predicting SOH ? 

# comparing LSTM with 2 nodes at output (SOC and SOH) vs. just 1 node (SOC or SOH)
# Y = df[['SOC', 'SOH']]
# Y = df[['SOC']]
# Y = df[['SOH']]
Y = df['SOH'].values

# Normalize input data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing
# x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)  # use normalized features to prevent overfitting (X_scaled)

# Reshape input data for LSTM
# x_train = x_train.values.reshape((x_train.shape[0], x_train.shape[1], 1))
# x_test = x_test.values.reshape((x_test.shape[0], x_test.shape[1], 1))


# Define function to create sequences
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length, :-1])  # Features (voltage, current, temperature, state of charge)
        y.append(data[i + seq_length, -1])     # Target variable (state of health)
    return np.array(X), np.array(y)

# Choose sequence length
seq_length = 10 

# Create sequences for training and testing data
X_train_seq, y_train_seq = create_sequences(x_train, seq_length)
X_test_seq, y_test_seq = create_sequences(x_test, seq_length)

# Print shapes of train and test data
print("X_train shape:", X_train_seq.shape)
print("y_train shape:", y_train_seq.shape)
print("X_test shape:", X_test_seq.shape)
print("y_test shape:", y_test_seq.shape)

df

X_train shape: (1312746, 10, 3)
y_train shape: (1312746,)
X_test shape: (328180, 10, 3)
y_test shape: (328180,)


,V,I,SOC,T_surf,SOH
0,4.200000,0.000000,100.000000,25.000000,100
1,4.198758,-0.225199,100.000000,25.000000,100
2,4.197524,-0.449161,100.000000,25.000000,100
3,4.196296,-0.671867,100.000000,25.000000,100
4,4.190443,-1.733425,100.000000,25.000000,100
...,...,...,...,...,...
1640941,3.040704,-41.000000,14.330671,95.558472,70
1640942,3.019722,-41.000000,13.112293,96.489709,70
1640943,2.985720,-41.000000,11.355930,97.840139,70
1640944,2.945987,-41.000000,9.599568,99.207108,70


# Fit Model

In [9]:
# create an LSTM to train our model for finding SOC/SOH

# Define model for predicting SOH or SOC 
model = Sequential()
# model.add(LSTM(50, activation='relu', input_shape=(x_train.shape[1], 1))) # for SOC or SOH 

# Defining a model for predicting SOH/SOC
# model.add(LSTM(50, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])))
# model.add(LSTM(50, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2]), kernel_regularizer=l2(0.01)))
model.add(LSTM(50, activation='relu', input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))

# model.add(Dense(2))  # Output layer for two continuous values: SOC and SOH
# model.add(Dense(1))  # Output layer for one continuous value: SOC or SOH
model.add(Dense(units=1))

# Compile model with Adam optimizer and custom learning rate
# opt = Adam(learning_rate=0.001)
# model.compile(optimizer=opt, loss='mse')  
model.compile(optimizer='adam', loss='mean_squared_error')

model.summary()

# Train the model
# model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test), verbose=2)
# model.fit(x_train, y_train, epochs=50, batch_size=32)
model.fit(X_train_seq, y_train_seq, epochs=10, batch_size=32, validation_data=(X_test_seq, y_test_seq))

# TODO: THINGS TO CONSIDER FOR FINE-TUNING HYPERPARAMETERS
# TODO: 1) consider altering the depth of the network. currently we have 1 hidden layer w/ 50 nodes 
# TODO: 2) consider altering the width of the network. current there are 50 nodes, but that could be fine-tuned 
# TODO: 3) consider altering the activation function. right now it is set to be relu 
# TODO: 4) consider altering the optimizier. right now it is set to Adam
# TODO: 5) consider altering the number of epochs 

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 50)                10800     
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 10851 (42.39 KB)
Trainable params: 10851 (42.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
41024/41024 [==============================] - 222s 5ms/step - loss: 0.1274 - val_loss: 0.1272
Epoch 2/10
41024/41024 [==============================] - 208s 5ms/step - loss: 0.1272 - val_loss: 0.1272
Epoch 3/10
41024/41024 [==============================] - 211s 5ms/step - loss: 0.1272 - val_loss: 0.1272
Epoch 4/10
41024/41024 [==============================] - 213s 5ms/step - loss: 0.1272 - val_los

# Test Model / Predict

In [13]:
# Reshape input data for LSTM
x_test_reshaped = X_test_seq.reshape((X_test_seq.shape[0], X_test_seq.shape[1], X_test_seq.shape[2]))

# Evaluate the model
loss = model.evaluate(x_test_reshaped, y_test_seq)
print("Test Loss:", loss)

# Make predictions
predictions = model.predict(x_test_reshaped)


10256/10256 [==============================] - 29s 3ms/step - loss: 0.1273
Test Loss: 0.1272556632757187
10256/10256 [==============================] - 34s 2ms/step


# Save Model and Export (Optimize for Size)